---
title: Tensorflow1 dataset
tags: 小书匠,tensorflow,tensorflow1,dataset,make_csv_dataset
grammar_cjkRuby: true
# renderNumberedHeading: true
---

[toc!]

# Tensorflow1 dataset

tensorflow1 和 tensorflow2 的 dataset 的用法略有不同，这里是 tensorflow1 中的 dataset 的用法


In [15]:
import tensorflow as tf

print(tf.__version__)

1.15.0


我们知道，在TensorFlow中可以使用feed-dict的方式输入数据信息，但是这种方法的速度是最慢的，在实际应用中应该尽量避免这种方法。而使用输入管道就可以保证GPU在工作时无需等待新的数据输入，这才是正确的方法。

幸运的是，TensorFlow提供了一种内置的API——Dataset，使得我们可以很容易地就利用输入管道的方式输入数据。在这篇教程中，我们将介绍如何创建和使用输入管道以及如何高效地向模型输入数据。

## 概述

使用Dataset的三个步骤：

1. 载入数据：为数据创建一个Dataset实例

2. 创建一个迭代器：使用创建的数据集来构造一个Iterator实例以遍历数据集

3. 使用数据：使用创建的迭代器，我们可以从数据集中获取数据元素，从而输入到模型中去。

许多 tensorflow 的高层 api 如 tf.keras，tf.estimator 中，2、3 步骤被封装起来，我们只需要完成第 1 步就好了

## 载入数据

首先，我们需要将一些数据放到数据集中。

### 从numpy载入

这是最常见的情况，假设我们有一个numpy数组，我们想将它传递给TensorFlow

In [24]:
# create a random vector of shape (100,2)
x = np.random.sample((100,2))
# make a dataset from a numpy array
dataset = tf.data.Dataset.from_tensor_slices(x)

我们也可以传递多个numpy数组，最典型的例子是当数据被划分为特征和标签的时候：

In [25]:
features, labels = (np.random.sample((100,2)), np.random.sample((100,1)))
dataset = tf.data.Dataset.from_tensor_slices((features,labels))

### 从tensors中载入

我们当然也可以用一些张量初始化数据集

In [26]:
# using a tensor
dataset = tf.data.Dataset.from_tensor_slices(tf.random_uniform([100, 2]))

### 从placeholder中载入

如果我们想动态地改变Dataset中的数据，使用这种方式是很有用的。

In [27]:
x = tf.placeholder(tf.float32, shape=[None,2])
dataset = tf.data.Dataset.from_tensor_slices(x)

### 从generator载入

我们也可以从generator中初始化一个Dataset。当一个数组中元素长度不相同时，使用这种方式处理是很有效的。（例如一个序列）

In [49]:
sequence = np.array([[1],[2,3],[3,4]])

def generator():
    for el in sequence:
        yield el

dataset = tf.data.Dataset.from_generator(generator,
                                           output_types=tf.float32,
                                           output_shapes=tf.TensorShape([]))

在这种情况下，你还需要指定数据的类型和大小以创建正确的tensor

另一个例子

In [22]:
import warnings
warnings.filterwarnings('ignore')
import tensorflow as tf
import functools
import csv

TRAIN_URL = "http://download.tensorflow.org/data/iris_training.csv"
TEST_URL = "http://download.tensorflow.org/data/iris_test.csv"

def downloadfiles():
    train_path = tf.keras.utils.get_file(fname=r'./data', origin=TRAIN_URL)
    test_path = tf.keras.utils.get_file(fname=r'./data', origin=TEST_URL)
    return train_path, test_path

train_path,test_path = downloadfiles()
print("train_path: {}\ntest_path: {}".format(train_path, test_path))

# 注意，这两个不能用 list, 而得是 tuple，否则会报错 unhashable type: 'list'
output_types = (tf.float32, tf.int32)
output_shapes = (tf.TensorShape([None]), tf.TensorShape([]))

def data_generator(file_path):
    with open(file_path) as f:
        reader = csv.reader(f)
        next(reader) # skip header
        for row in reader:
            label = row.pop()
            features = row
            yield features, label

dataset = tf.data.Dataset.from_generator(
        functools.partial(data_generator, train_path), 
        output_types=output_types,
        output_shapes=output_shapes
)
iterator = dataset.make_one_shot_iterator()
next_batch = iterator.get_next()

with tf.Session() as sess:
    try:
        while True:
            print(sess.run(next_batch))
    except tf.errors.OutOfRangeError:
        print("end")

train_path: /Users/ed/.keras/datasets/./data
test_path: /Users/ed/.keras/datasets/./data
(array([6.4, 2.8, 5.6, 2.2], dtype=float32), 2)
(array([5. , 2.3, 3.3, 1. ], dtype=float32), 1)
(array([4.9, 2.5, 4.5, 1.7], dtype=float32), 2)
(array([4.9, 3.1, 1.5, 0.1], dtype=float32), 0)
(array([5.7, 3.8, 1.7, 0.3], dtype=float32), 0)
(array([4.4, 3.2, 1.3, 0.2], dtype=float32), 0)
(array([5.4, 3.4, 1.5, 0.4], dtype=float32), 0)
(array([6.9, 3.1, 5.1, 2.3], dtype=float32), 2)
(array([6.7, 3.1, 4.4, 1.4], dtype=float32), 1)
(array([5.1, 3.7, 1.5, 0.4], dtype=float32), 0)
(array([5.2, 2.7, 3.9, 1.4], dtype=float32), 1)
(array([6.9, 3.1, 4.9, 1.5], dtype=float32), 1)
(array([5.8, 4. , 1.2, 0.2], dtype=float32), 0)
(array([5.4, 3.9, 1.7, 0.4], dtype=float32), 0)
(array([7.7, 3.8, 6.7, 2.2], dtype=float32), 2)
(array([6.3, 3.3, 4.7, 1.6], dtype=float32), 1)
(array([6.8, 3.2, 5.9, 2.3], dtype=float32), 2)
(array([7.6, 3. , 6.6, 2.1], dtype=float32), 2)
(array([6.4, 3.2, 5.3, 2.3], dtype=float32), 2)

### 从 csv 文件中创建

In [1]:
import warnings
warnings.filterwarnings('ignore')
import tensorflow as tf
import functools
import csv

titanic_file = tf.keras.utils.get_file("train.csv", "https://storage.googleapis.com/tf-datasets/titanic/train.csv")
print("titanic_file: ", titanic_file)

dataset = tf.data.experimental.make_csv_dataset(titanic_file, 
                                                batch_size=4, 
                                                label_name="survived")

iterator = dataset.make_one_shot_iterator()
next_batch = iterator.get_next()

with tf.Session() as sess:
    print(sess.run(next_batch))

titanic_file:  /Users/ed/.keras/datasets/train.csv
Instructions for updating:
Use `tf.data.Dataset.interleave(map_func, cycle_length, block_length, num_parallel_calls=tf.data.experimental.AUTOTUNE)` instead. If sloppy execution is desired, use `tf.data.Options.experimental_determinstic`.
Instructions for updating:
Use `tf.data.Dataset.shuffle(buffer_size, seed)` followed by `tf.data.Dataset.repeat(count)`. Static tf.data optimizations will take care of using the fused implementation.
Instructions for updating:
Use `for ... in dataset:` to iterate over a dataset. If using `tf.estimator`, return the `Dataset` object directly from your input function. As a last resort, you can use `tf.compat.v1.data.make_one_shot_iterator(dataset)`.
(OrderedDict([('sex', array([b'male', b'male', b'male', b'male'], dtype=object)), ('age', array([32., 21., 30., 24.], dtype=float32)), ('n_siblings_spouses', array([0, 0, 0, 2], dtype=int32)), ('parch', array([0, 0, 0, 0], dtype=int32)), ('fare', array([ 7.925

#### tf.data.experimental.make_csv_dataset

## 创建一个迭代器

我们已经知道了如何创建数据集，但是如何从中获取数据呢？我们需要使用一个Iterator遍历数据集并重新得到数据真实值。有四种形式的迭代器。

### One shot Iterator

这是最简单的迭代器，下面给出第一个例子：

In [53]:
x = np.random.sample((10, 2))

# make a dataset from a numpy array
dataset = tf.data.Dataset.from_tensor_slices(x)

# create the iterator
iterator = dataset.make_one_shot_iterator()

# 调用get_next()来获得包含数据的张量
next_batch = iterator.get_next()

with tf.Session() as sess:
    try:
        while True:
            ret = sess.run(next_batch)
            print(ret)
    except tf.errors.OutOfRangeError:
        print("end")

[0.76747849 0.04158681]
[0.8919353  0.63975712]
[0.74729546 0.70326269]
[0.66525039 0.51324041]
[0.37313983 0.3916572 ]
[0.30910254 0.59993772]
[0.59367427 0.22685718]
[0.41151761 0.08332528]
[0.45145689 0.73832339]
[0.12102829 0.01189038]
end


### 可初始化的迭代器

如果我们想建立一个可以在运行时改变数据源的动态数据集，我们可以用placeholder 创建一个数据集。接着用常见的feed-dict机制初始化这个placeholder。这些工作可以通过使用一个可初始化的迭代器完成。使用上一节的第三个例子

In [58]:

data = np.random.sample((100,2))
# using a placeholder
x = tf.placeholder(tf.float32, shape=[None,2])
dataset = tf.data.Dataset.from_tensor_slices(x)
iterator = dataset.make_initializable_iterator() # create the iteratorator
next_batch = iterator.get_next()
with tf.Session() as sess:
    # feed the placeholder with data
    sess.run(iterator.initializer, feed_dict={ x: data })
    print(sess.run(next_batch)) # output [ 0.52374458  0.71968478]

[0.9798317 0.1434286]


In [ ]:
# initializable iterator to switch between dataset
EPOCHS = 10
x, y = tf.placeholder(tf.float32, shape=[None,2]), tf.placeholder(tf.float32, shape=[None,1])
dataset = tf.data.Dataset.from_tensor_slices((x, y))
train_data = (np.random.sample((100,2)), np.random.sample((100,1)))
test_data = (np.array([[1,2]]), np.array([[0]]))
iter = dataset.make_initializable_iterator()
features, labels = iter.get_next()
with tf.Session() as sess:
#     initialise iterator with train data
    sess.run(iter.initializer, feed_dict={ x: train_data[0], y: train_data[1]})
    for _ in range(EPOCHS):
        sess.run([features, labels])
#     switch to test data
    sess.run(iter.initializer, feed_dict={ x: test_data[0], y: test_data[1]})
    print(sess.run([features, labels]))

In [ ]:
### 可重新初始化的迭代器

这个概念和之前的相似，我们想在数据间动态切换。但是我们是转换数据集而不是把新数据送到相同的数据集。和之前一样，我们需要一个训练集和一个测试集

# making fake data using numpy
train_data = (np.random.sample((100,2)), np.random.sample((100,1)))
test_data = (np.random.sample((10,2)), np.random.sample((10,1)))

接下来创建两个Dataset

# create two datasets, one for training and one for test
train_dataset = tf.data.Dataset.from_tensor_slices(train_data)
test_dataset = tf.data.Dataset.from_tensor_slices(test_data)

现在我们要用到一个小技巧，即创建一个通用的Iterator

# create a iterator of the correct shape and type
iter = tf.data.Iterator.from_structure(train_dataset.output_types,
                                           train_dataset.output_shapes)

接着创建两个初始化运算

# create the initialisation operations
train_init_op = iter.make_initializer(train_dataset)
test_init_op = iter.make_initializer(test_dataset)

和之前一样，我们得到下一个元素

features, labels = iter.get_next()

现在，我们可以直接使用session运行两个初始化运算。把上面这些综合起来我们可以得到：

# Reinitializable iterator to switch between Datasets
EPOCHS = 10
# making fake data using numpy
train_data = (np.random.sample((100,2)), np.random.sample((100,1)))
test_data = (np.random.sample((10,2)), np.random.sample((10,1)))
# create two datasets, one for training and one for test
train_dataset = tf.data.Dataset.from_tensor_slices(train_data)
test_dataset = tf.data.Dataset.from_tensor_slices(test_data)
# create a iterator of the correct shape and type
iter = tf.data.Iterator.from_structure(train_dataset.output_types,
                                           train_dataset.output_shapes)
features, labels = iter.get_next()
# create the initialisation operations
train_init_op = iter.make_initializer(train_dataset)
test_init_op = iter.make_initializer(test_dataset)
with tf.Session() as sess:
    sess.run(train_init_op) # switch to train dataset
    for _ in range(EPOCHS):
        sess.run([features, labels])
    sess.run(test_init_op) # switch to val dataset
    print(sess.run([features, labels]))

## 使用数据

在之前的例子中，我们使用session来打印Dataset中next元素的值

```
...
next_el = iter.get_next()
...
print(sess.run(next_el)) # will output the current element
```

现在为了向模型传递数据，我们只需要传递get_next()产生的张量。

在下面的代码中，我们有一个包含两个numpy数组的Dataset，这里用到了和第一节一样的例子。注意到我们需要将.random.sample封装到另外一个numpy数组中，因此会增加一个维度以用于数据batch。

In [41]:
EPOCHS = 10
BATCH_SIZE = 16
# using two numpy arrays
features, labels = (np.array([np.random.sample((100,2))]),
                    np.array([np.random.sample((100,1))]))
dataset = tf.data.Dataset.from_tensor_slices((features,labels)).repeat().batch(BATCH_SIZE)
iterator = dataset.make_one_shot_iterator()
x, y = iterator.get_next()

# make a simple model
net = tf.layers.dense(x, 8, activation=tf.tanh) # pass the first value from iter.get_next() as input
net = tf.layers.dense(net, 8, activation=tf.tanh)
prediction = tf.layers.dense(net, 1, activation=tf.tanh)
loss = tf.losses.mean_squared_error(prediction, y) # pass the second value from iter.get_net() as label
train_op = tf.train.AdamOptimizer().minimize(loss)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(EPOCHS):
        _, loss_value = sess.run([train_op, loss])
        print("Iter: {}, Loss: {:.4f}".format(i, loss_value))

Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Iter: 0, Loss: 0.4063
Iter: 1, Loss: 0.3880
Iter: 2, Loss: 0.3702
Iter: 3, Loss: 0.3531
Iter: 4, Loss: 0.3366
Iter: 5, Loss: 0.3208
Iter: 6, Loss: 0.3056
Iter: 7, Loss: 0.2911
Iter: 8, Loss: 0.2773
Iter: 9, Loss: 0.2641


## 有用的技巧

### batch

通常情况下，batch是一件麻烦的事情，但是通过Dataset API我们可以使用batch(BATCH_SIZE)方法自动地将数据按照指定的大小batch，默认值是1。在接下来的例子中，我们使用的batch大小为4。

In [44]:
# BATCHING
BATCH_SIZE = 4
x = np.random.sample((100,2))
# make a dataset from a numpy array
dataset = tf.data.Dataset.from_tensor_slices(x).batch(BATCH_SIZE)

iterator = dataset.make_one_shot_iterator()
el = iterator.get_next()

with tf.Session() as sess:
    print(sess.run(el))

[[1.73756691e-01 4.69362655e-01]
 [9.67855623e-01 3.44047169e-01]
 [5.17740357e-01 2.13477521e-01]
 [7.90970741e-04 9.16710425e-01]]


### padded_batch

In [21]:
import tensorflow as tf
import numpy as np

tf.reset_default_graph()

x = [[1, 0, 0], [2, 3, 0], [4, 5, 5], [7, 8, 0], [9, 0, 0], [0, 1, 0]]

#tf.TensorShape([])     表示长度为单个数字
#tf.TensorShape([None]) 表示不 padding
padded_shapes = tf.TensorShape([4]) # 指将一个 instance padding 成形状为 [4] 的

dataset = tf.data.Dataset.from_tensor_slices(x)
dataset = dataset.padded_batch(2, padded_shapes=padded_shapes)
iterator = dataset.make_one_shot_iterator()
next_batch = iterator.get_next()

with tf.Session() as sess:
    try:
        while True:
            print(sess.run(next_batch))
    except tf.errors.OutOfRangeError:
        print("end")

[[1 0 0 0]
 [2 3 0 0]]
[[4 5 5 0]
 [7 8 0 0]]
[[9 0 0 0]
 [0 1 0 0]]
end


### repeat

使用.repeat()我们可以指定数据集迭代的次数。如果没有设置参数，则迭代会一直循环。通常来说，一直循环并直接用标准循环控制epoch的次数能取得较好的效果。

### shuffle

我们可以使用shuffle()方法将Dataset随机洗牌，默认是在数据集中对每一个epoch洗牌，这种处理可以避免过拟合。

我们也可以设置buffer_size参数，下一个元素将从这个固定大小的缓存中按照均匀分布抽取。例子：

### map

你可以使用map()方法对数据集的每个成员应用自定义的函数。在下面的例子中，我们将每个元素乘以2。

In [50]:
# MAP

def func(x): # func 接受的是一个 Tensor
    return x * 2

x = np.array([[1],[2],[3],[4]])
# make a dataset from a numpy array
dataset = tf.data.Dataset.from_tensor_slices(x)
dataset = dataset.map(func)
iter = dataset.make_one_shot_iterator()
el = iter.get_next()
with tf.Session() as sess:
    for _ in range(len(x)):
        print(sess.run(el))

[2]
[4]
[6]
[8]


需要注意的是，func 的参数是一个 Tensor 类型，因此，我们只能在这里进行一些 Tensor 类型才支持的操作

# References
1. http://localhost:8888/lab/workspaces/auto-i/tree/DL-Project/learnTensorflow/Tensorflow%20dataset/Tensorflow1%20dataset.ipynb
2. [TensorFlow学习笔记(4): Tensorflow tf.data.Dataset - 知乎](https://zhuanlan.zhihu.com/p/37106443)
3. [tf.data: Build TensorFlow input pipelines  |  TensorFlow Core](https://www.tensorflow.org/guide/data#consuming_csv_da2a3)
4. [tf.data.experimental.make_csv_dataset  |  TensorFlow Core v2.4.1](https://www.tensorflow.org/api_docs/python/tf/data/experimental/make_csv_dataset)